In [3]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, tnrange

from sklearn.cross_decomposition import CCA

def cal_correlation_coefficient(corr,mode='max',abs=False):
    import numpy as np
    _list_mode = ['max','sum']
    if(type(corr) != type(np.array([]))): corr = np.array(corr)
    if(mode not in _list_mode): raise ValueError(f"mode can only be {_list_mode}")
    if(abs): corr = np.abs(corr)
    # print(corr)
    if(mode == 'max'): return corr.max(axis=1)
    if(mode == 'sum'): return corr.sum(axis=1)

corr_mode = 'sum'
corr_abs = True
# cal_correlation_coefficient(R[:X_chs,X_chs:X_chs+Y_chs],mode='sum',abs=False)


# Use all default parameter of FastICA
import pyroomacoustics as pra

## Prepare one-shot STFT
L = 2048
hop = L // 2
win_a = pra.hann(L)
win_s = pra.transform.stft.compute_synthesis_window(win_a, hop)

# Now we want to perform FastICA on all EEG data we have
# We have 4 EEG signals with 16 channels and each consists of 7 components (bands) + original + highpass = 9
states = ['deep-anesthetic','low-anesthetic','recovery','rest']
components = ['','_highpass','_0_delta_theta','_1_alpha','_2_low_beta','_3_high_beta','_4_low_gamma','_5_middle_gamma','_6_high_gamma']
for state in tqdm(states, desc='states'):
    mat_eeg = scipy.io.loadmat(f'EEG/EEG_{state}.mat')
    mat_ecog = scipy.io.loadmat(f'EEG/ECoG_{state}.mat')
    # RUN FastICA on original, highpass, each component
    for component in tqdm(components,desc='components'):
        eeg = []
        for i in range(16):
            eeg.append(mat_eeg[f'{i}{component}'][0])
        eeg = np.array(eeg)

        ecog = []
        for i in range(128):
            ecog.append(mat_ecog[f'{i}{component}'][0])
        ecog = np.array(ecog)
        # print(eeg.shape) (16,300000)
        # print(ecog.shape) (128,300000)
        # (n_features, n_samples)

        ## STFT ANALYSIS
        # (n_samples, n_channels)
        X = pra.transform.stft.analysis(eeg.T, L, hop, win=win_a)

        # Run ILRMA
        Y = pra.bss.ilrma(X, n_iter=100, n_components=16, proj_back=True) #callback=convergence_callback)
        transformed_eeg = pra.transform.stft.synthesis(Y, L, hop, win=win_s)
        # print(transformed_eeg.shape)
        # (300032, 16)
        transformed_eeg = transformed_eeg[:300000, :]
        # (300000, 16)

        # (n_samples, n_components)
        assert (eeg == transformed_eeg.T).all() == False, f"You data is the same."

        R = np.corrcoef(transformed_eeg.T, ecog)
        corr = cal_correlation_coefficient(R[:16,16:16+128],mode=corr_mode,abs=corr_abs)
        # print(corr.shape) (16,)
        # Step4
        # argsort is acsding. [::-1] will revert the result
        group = []
        EVs = []
        for idx in tqdm(corr.argsort()[::-1],desc='EVs'):
            group.append(idx)
            # print(transformed_eeg.T[group].shape)
            # For each subset of EEG components, applied CCA
            ev_cca = CCA(n_components=len(group))
            ev_cca.fit(transformed_eeg.T[group].T,ecog.T)
            ev_R = np.corrcoef(ev_cca.x_scores_.T,ev_cca.y_scores_.T)
            # print(ev_R.shape)
            # print(ev_R)
            # the "sum of canonical correlation coefficients" was calculated for each freq, state, and subject
            ccc = cal_correlation_coefficient(ev_R[:len(group),len(group):2*len(group)],mode='sum',abs=True)
            EVs.append(ccc)
            # print(EVs)
            # if(len(group) == 16): break
        # print(EVs)
        scipy.io.savemat(f'BSS/ILRMA_{state}{component}.mat', dict({'EVs': EVs}))
        # break
    # break
